Agents

Agents are the core building block in your apps. An agent is a large language model (LLM), configured with instructions and tools.

Basic configuration

The most common properties of an agent you'll configure are:

* name: A required string that identifies your agent.
* instructions: also known as a developer message or system prompt.
* model: which LLM to use, and optional model_settings to configure model tuning parameters like
  temperature, top_p, etc.
* tools: Tools that the agent can use to achieve its tasks.

ایجنٹس

ایجنٹ آپ کی ایپس میں بنیادی تعمیراتی بلاک ہیں۔ ایجنٹ ایک بڑی لینگویج ماڈل (ایل ایل ایم) ہے، جسے ہدایات اور ٹولز کے ساتھ ترتیب دیا گیا ہے۔

بنیادی ترتیب

ایک ایجنٹ کی سب سے عام خصوصیات جو آپ ترتیب دیں گے یہ ہیں:

* نام: ایک مطلوبہ تار جو آپ کے ایجنٹ کی شناخت کرتا ہے۔
* ہدایات: ڈویلپر پیغام یا سسٹم پرامپٹ کے نام سے بھی جانا جاتا ہے۔
* ماڈل: کون سا ایل ایل ایم استعمال کرنا ہے، اور ماڈل ٹیوننگ پیرامیٹرز کو ترتیب دینے کے لیے اختیاری model_settings جیسے 
درجہ حرارت، top_p، وغیرہ
* ٹولز: وہ ٹولز جنہیں ایجنٹ اپنے کاموں کو حاصل کرنے کے لیے استعمال کر سکتا ہے۔

In [ ]:
from agents import Agent, ModelSettings, function_tool

@function_tool
def get_weather(city: str) -> str:
     """returns weather info for the specified city."""
    return f"The weather in {city} is sunny"

agent = Agent(
    name="Haiku agent",
    instructions="Always respond in haiku form",
    model="o3-mini",
    tools=[get_weather],
)

Context

Agents are generic on their context type. Context is a dependency-injection tool: it's an object you create and pass to Runner.run(), that is passed to every agent, tool, handoff etc, and it serves as a grab bag of dependencies and state for the agent run. You can provide any Python object as the context.

سیاق و سباق

ایجنٹ اپنے سیاق و سباق کی قسم پر عام ہوتے ہیں۔ سیاق و سباق ایک انحصاری انجیکشن ٹول ہے: یہ ایک ایسی چیز ہے جسے آپ بناتے ہیں اور Runner.run() کو پاس کرتے ہیں، جو ہر ایجنٹ، ٹول، ہینڈ آف وغیرہ کو دیا جاتا ہے، اور یہ ایجنٹ چلانے کے لیے انحصار اور ریاست کے گراب بیگ کے طور پر کام کرتا ہے۔ آپ سیاق و سباق کے طور پر کوئی بھی ازگر آبجیکٹ فراہم کر سکتے ہیں۔

In [ ]:
@dataclass
class UserContext:
    name: str
    uid: str
    is_pro_user: bool

    async def fetch_purchases() -> list[Purchase]:
        return ...

agent = Agent[UserContext](
    ...,
)

Output types

By default, agents produce plain text (i.e. str) outputs. If you want the agent to produce a particular type of output, you can use the output_type parameter. A common choice is to use Pydantic objects, but we support any type that can be wrapped in a Pydantic TypeAdapter - dataclasses, lists, TypedDict, etc.

آؤٹ پٹ کی اقسام

پہلے سے طے شدہ طور پر، ایجنٹ سادہ متن (یعنی str) آؤٹ پٹ تیار کرتے ہیں۔ اگر آپ چاہتے ہیں کہ ایجنٹ ایک خاص قسم کی آؤٹ پٹ پیدا کرے، تو آپ output_type پیرامیٹر استعمال کر سکتے ہیں۔ ایک عام انتخاب Pydantic اشیاء کا استعمال کرنا ہے، لیکن ہم کسی بھی قسم کی حمایت کرتے ہیں جسے Pydantic TypeAdapter میں لپیٹا جا سکتا ہے - ڈیٹا کلاسز، فہرستیں، TypedDict، وغیرہ۔

In [ ]:
from pydantic import BaseModel
from agents import Agent


class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

agent = Agent(
    name="Calendar extractor",
    instructions="Extract calendar events from text",
    output_type=CalendarEvent,
)

Handoffs

Handoffs are sub-agents that the agent can delegate to. You provide a list of handoffs, and the agent can choose to delegate to them if relevant. This is a powerful pattern that allows orchestrating modular, specialized agents that excel at a single task. 

ہینڈ آف

ہینڈ آف ذیلی ایجنٹ ہیں جن کو ایجنٹ تفویض کر سکتا ہے۔ آپ ہینڈ آف کی فہرست فراہم کرتے ہیں، اور اگر متعلقہ ہو تو ایجنٹ ان کو تفویض کرنے کا انتخاب کر سکتا ہے۔ یہ ایک طاقتور نمونہ ہے جو ماڈیولر، خصوصی ایجنٹوں کو آرکیسٹریٹنگ کی اجازت دیتا ہے جو ایک ہی کام میں سبقت لے جاتے ہیں۔

In [ ]:
from agents import Agent

booking_agent = Agent(...)
refund_agent = Agent(...)

triage_agent = Agent(
    name="Triage agent",
    instructions=(
        "Help the user with their questions."
        "If they ask about booking, handoff to the booking agent."
        "If they ask about refunds, handoff to the refund agent."
    ),
    handoffs=[booking_agent, refund_agent],
)

Dynamic instructions

In most cases, you can provide instructions when you create the agent. However, you can also provide dynamic instructions via a function. The function will receive the agent and context, and must return the prompt. Both regular and async functions are accepted.

متحرک ہدایات

زیادہ تر معاملات میں، آپ ایجنٹ بناتے وقت ہدایات فراہم کر سکتے ہیں۔ تاہم، آپ ایک فنکشن کے ذریعے متحرک ہدایات بھی فراہم کر سکتے ہیں۔ فنکشن ایجنٹ اور سیاق و سباق وصول کرے گا، اور اسے فوری طور پر واپس کرنا ہوگا۔ باقاعدہ اور async دونوں فنکشنز کو قبول کیا جاتا ہے۔

In [ ]:
def dynamic_instructions(
    context: RunContextWrapper[UserContext], agent: Agent[UserContext]
) -> str:
    return f"The user's name is {context.context.name}. Help them with their questions."


agent = Agent[UserContext](
    name="Triage agent",
    instructions=dynamic_instructions,
)

Lifecycle events (hooks)

Sometimes, you want to observe the lifecycle of an agent. For example, you may want to log events, or pre-fetch data when certain events occur. You can hook into the agent lifecycle with the hooks property.

لائف سائیکل واقعات (ہکس)

کبھی کبھی، آپ ایک ایجنٹ کے لائف سائیکل کا مشاہدہ کرنا چاہتے ہیں۔ مثال کے طور پر، آپ واقعات کو لاگ کرنا چاہتے ہیں، یا کچھ واقعات پیش آنے پر ڈیٹا کو پہلے سے بازیافت کرنا چاہتے ہیں۔ آپ ہکس پراپرٹی کے ساتھ ایجنٹ کے لائف سائیکل کو جوڑ سکتے ہیں۔

Guardrails

Guardrails allow you to run checks/validations on user input in parallel to the agent running, and on the agent's output once it is produced. For example, you could screen the user's input and agent's output for relevance.

گارڈریلز

گارڈریلز آپ کو صارف کے ان پٹ پر ایجنٹ کے چلنے کے متوازی طور پر چیک/توثیق چلانے کی اجازت دیتے ہیں، اور ایک بار جب یہ تیار ہو جاتا ہے تو ایجنٹ کے آؤٹ پٹ پر۔ مثال کے طور پر، آپ مطابقت کے لیے صارف کے ان پٹ اور ایجنٹ کے آؤٹ پٹ کو اسکرین کر سکتے ہیں۔

Cloning / Copying Agents

By using the clone() method on an agent, you can duplicate an Agent, and optionally change any properties you like.

کلوننگ / کاپی کرنے والے ایجنٹ

ایک ایجنٹ پر کلون() طریقہ استعمال کرکے، آپ ایجنٹ کی نقل بنا سکتے ہیں، اور اختیاری طور پر اپنی پسند کی کوئی بھی خصوصیات تبدیل کر سکتے ہیں۔

In [ ]:
pirate_agent = Agent(
    name="Pirate",
    instructions="Write like a pirate",
    model="o3-mini",
)

robot_agent = pirate_agent.clone(
    name="Robot",
    instructions="Write like a robot",
)

Forcing tool use

Supplying a list of tools doesn't always mean the LLM will use a tool. You can force tool use by setting ModelSettings.tool_choice. Valid values are:

1. auto, which allows the LLM to decide whether or not to use a tool.
2. required, which requires the LLM to use a tool (but it can intelligently decide which tool).
3. none, which requires the LLM to not use a tool.
4. Setting a specific string e.g. my_tool, which requires the LLM to use that specific tool.

زبردستی ٹول استعمال کرنا

ٹولز کی فہرست فراہم کرنے کا مطلب ہمیشہ یہ نہیں ہوتا ہے کہ LLM ٹول استعمال کرے گا۔ آپ ModelSettings.tool_choice سیٹ کر کے ٹول کے استعمال پر مجبور کر سکتے ہیں۔ درست اقدار ہیں:

1. آٹو، جو LLM کو ٹول استعمال کرنے یا نہ کرنے کا فیصلہ کرنے کی اجازت دیتا ہے۔
2. درکار ہے، جس کے لیے LLM کو ٹول استعمال کرنے کی ضرورت ہے (لیکن یہ ذہانت سے فیصلہ کر سکتا ہے کہ کون سا ٹول ہے)۔
3. کوئی نہیں، جس کے لیے LLM کو ٹول استعمال نہ کرنے کی ضرورت ہو۔
4. ایک مخصوص سٹرنگ ترتیب دینا جیسے my_tool، جس میں LLM کو اس مخصوص ٹول کو استعمال کرنے کی ضرورت ہوتی ہے۔

In [ ]:
from agents import Agent, Runner, function_tool, ModelSettings

@function_tool
def get_weather(city: str) -> str:
    """Returns weather info for the specified city."""
    return f"The weather in {city} is sunny"

agent = Agent(
    name="Weather Agent",
    instructions="Retrieve weather details.",
    tools=[get_weather],
    model_settings=ModelSettings(tool_choice="get_weather") 
)

Tool Use Behavior

The tool_use_behavior parameter in the Agent configuration controls how tool outputs are handled: - "run_llm_again": The default. Tools are run, and the LLM processes the results to produce a final response. - "stop_on_first_tool": The output of the first tool call is used as the final response, without further LLM processing.

ٹول کے استعمال کا برتاؤ

ایجنٹ کنفیگریشن میں ٹول_استعمال_بی ہیویور پیرامیٹر کنٹرول کرتا ہے کہ ٹول آؤٹ پٹس کو کیسے ہینڈل کیا جاتا ہے: - "run_llm_again": ڈیفالٹ۔ ٹولز چلائے جاتے ہیں، اور LLM حتمی ردعمل پیدا کرنے کے لیے نتائج پر کارروائی کرتا ہے۔ - "stop_on_first_tool": پہلے ٹول کال کی آؤٹ پٹ کو حتمی جواب کے طور پر استعمال کیا جاتا ہے، مزید LLM پروسیسنگ کے بغیر۔

In [ ]:
from agents import Agent, Runner, function_tool, ModelSettings

@function_tool
def get_weather(city: str) -> str:
    """Returns weather info for the specified city."""
    return f"The weather in {city} is sunny"

agent = Agent(
    name="Weather Agent",
    instructions="Retrieve weather details.",
    tools=[get_weather],
    tool_use_behavior="stop_on_first_tool"
)

* StopAtTools(stop_at_tool_names=[...]): Stops if any specified tool is called, using its output as the
  final response.

  * StopAtTools(stop_at_tool_names=[...]): اگر کسی مخصوص ٹول کو کال کیا جائے تو اس کے آؤٹ پٹ کو بطور استعمال کرتے ہوئے رک جاتا ہے۔ 
حتمی جواب.

In [ ]:
from agents import Agent, Runner, function_tool
from agents.agent import StopAtTools

@function_tool
def get_weather(city: str) -> str:
    """Returns weather info for the specified city."""
    return f"The weather in {city} is sunny"

@function_tool
def sum_numbers(a: int, b: int) -> int:
    """Adds two numbers."""
    return a + b

agent = Agent(
    name="Stop At Stock Agent",
    instructions="Get weather or sum numbers.",
    tools=[get_weather, sum_numbers],
    tool_use_behavior=StopAtTools(stop_at_tool_names=["get_weather"])
)

* ToolsToFinalOutputFunction: A custom function that processes tool results and decides whether to stop
  or continue with the LLM.

  * ToolsToFinalOutputFunction: ایک حسب ضرورت فنکشن جو ٹول کے نتائج پر کارروائی کرتا ہے اور فیصلہ کرتا ہے کہ آیا رکنا ہے 
یا LLM کے ساتھ جاری رکھیں۔

In [ ]:
from agents import Agent, Runner, function_tool, FunctionToolResult, RunContextWrapper
from agents.agent import ToolsToFinalOutputResult
from typing import List, Any

@function_tool
def get_weather(city: str) -> str:
    """Returns weather info for the specified city."""
    return f"The weather in {city} is sunny"

def custom_tool_handler(
    context: RunContextWrapper[Any],
    tool_results: List[FunctionToolResult]
) -> ToolsToFinalOutputResult:
    """Processes tool results to decide final output."""
    for result in tool_results:
        if result.output and "sunny" in result.output:
            return ToolsToFinalOutputResult(
                is_final_output=True,
                final_output=f"Final weather: {result.output}"
            )
    return ToolsToFinalOutputResult(
        is_final_output=False,
        final_output=None
    )

agent = Agent(
    name="Weather Agent",
    instructions="Retrieve weather details.",
    tools=[get_weather],
    tool_use_behavior=custom_tool_handler
)

To prevent infinite loops, the framework automatically resets tool_choice to "auto" after a tool call. This behavior is configurable via agent.reset_tool_choice. The infinite loop is because tool results are sent to the LLM, which then generates another tool call because of tool_choice, ad infinitum.

لامحدود لوپس کو روکنے کے لیے، فریم ورک ٹول کال کے بعد ٹول_چوائس کو خود بخود "آٹو" پر ری سیٹ کر دیتا ہے۔ یہ سلوک agent.reset_tool_choice کے ذریعے قابل ترتیب ہے۔ لامحدود لوپ اس لیے ہے کہ ٹول کے نتائج LLM کو بھیجے جاتے ہیں، جو پھر tool_choice، ad infinitum کی وجہ سے ایک اور ٹول کال تیار کرتا ہے۔